## Creating Function Blocks with reasoning

For simplicity we are focussing on the air temperatures in the AHU. Let us assume that we have an minimal Brick ontology that defines an Air Handling Unit AHU1 with four points.
```
Outside Air Temperature Sensor
Return Air Temperature Sensor
Discharge Air Temperature Sensor
Discharge Air Temperature Setpoint
```

In [1]:
import rdflib
g = rdflib.Graph()
src = '''
@prefix bf:  <http://buildsys.org/ontologies/BrickFrame#> .
@prefix ts:  <http://buildsys.org/ontologies/Brick#> .
@prefix :    <http://buildsys.org/ontologies/Example/AHU_FunctionBlock#> .

:AHU1 a           ts:AHU.
:AHU1 bf:hasPoint :OAT.
:AHU1 bf:hasPoint :AHU1RAT.
:AHU1 bf:hasPoint :AHU1DAT.
:AHU1 bf:hasPoint :AHU1DATSP.

:OAT        a  bf:Point, ts:Outside_Air_Temperature_Sensor. 
:AHU1RAT    a  bf:Point, ts:Return_Air_Temperature_Sensor. 
:AHU1DAT    a  bf:Point, ts:Discharge_Air_Temperature_Sensor. 
:AHU1DATSP  a  bf:Point, ts:Discharge_Air_Temperature_Setpoint.
'''
result = g.parse(data=src, format='n3')
print(len(g))

13


From this simple ontology we want to create a function block representing the following AHU Flow Graph
![AUH Flow](demo_function_block.png)


We can do this using SPARQL update (SPARUL). SPARUL consists of a WHERE query like SPARQL to define a graph pattern that is selected and an INSERT block to modify the graph around the selections.

### Creating the AHU Function Block

It is possible to create a large SPARUD to create the whole flow graph as shown in the figure in one query. We are using a more generic approach that creates the elements  individually and is still applicable if parts are missing if for example some points are not available.

Let us first classify the points as input and output points. Therefore, we are applying the following implication writen in SWRL
~~~~
AHU(?ahu) ∧ Discharge_Air_Temperature_Sensor(?point) ∧ hasPoint(?ahu,?point) ⇒ hasOutput(?ahu,?point)
~~~~
which is written in SPARUD as 

In [2]:
qres = g.update('''
PREFIX  bf:  <http://buildsys.org/ontologies/BrickFrame#>
PREFIX  ts:  <http://buildsys.org/ontologies/Brick#>
INSERT {
  ?ahu   bf:hasOutput ?point.
} WHERE {
  ?ahu   a ts:AHU.
  ?ahu   bf:hasPoint ?point.
  ?point a ts:Discharge_Air_Temperature_Sensor.
}
''')
print(len(g))

14


To validate the execution lets check if the tripple was created.

In [3]:
qres = g.query('''
PREFIX  bf:  <http://buildsys.org/ontologies/BrickFrame#>
PREFIX  ts:  <http://buildsys.org/ontologies/Brick#>
SELECT ?ahu ?point {
  ?ahu   bf:hasOutput ?point.
}
''')
for r in qres:
  print(r)

(rdflib.term.URIRef('http://buildsys.org/ontologies/Example/AHU_FunctionBlock#AHU1'), rdflib.term.URIRef('http://buildsys.org/ontologies/Example/AHU_FunctionBlock#AHU1DAT'))


We can define something similar for the inputs.
~~~~
ts:AHU(?ahu) ∧ ts:Return_Air_Temperature_Sensor(?point)  ∧ bf:hasPoint(?ahu,?point) ⇒ bf:hasInput(?ahu,?point)
ts:AHU(?ahu) ∧ ts:Outside_Air_Temperature_Sensor(?point) ∧ bf:hasPoint(?ahu,?point) ⇒ bf:hasInput(?ahu,?point)
ts:AHU(?ahu) ∧ ts:Discharge_Air_Temperature_Setpoint(?point) ∧ bf:hasPoint(?ahu,?point) ⇒ bf:hasInput(?ahu,?point)
~~~~

In [4]:
qres = g.update('''
PREFIX  bf:  <http://buildsys.org/ontologies/BrickFrame#>
PREFIX  ts:  <http://buildsys.org/ontologies/Brick#>
INSERT {
  ?ahu   bf:hasInput ?point.
} WHERE {
  ?ahu   a ts:AHU.
  ?ahu   bf:hasPoint ?point.
  {?point a ts:Return_Air_Temperature_Sensor.}
  UNION
  {?point a ts:Outside_Air_Temperature_Sensor.}
  UNION
  {?point a ts:Discharge_Air_Temperature_Setpoint.}
}
''')
print(len(g))

17


Now we define a FunctionBlock as an element that has inputs or outputs.
~~~~
bf:hasInput(?ahu,?point)  ⇒ bf:FunctionBlock(?ahu)
bf:hasOutput(?ahu,?point) ⇒ bf:FunctionBlock(?ahu)
~~~~

In [5]:
qres = g.update('''
PREFIX  bf:  <http://buildsys.org/ontologies/BrickFrame#>
INSERT {
  ?ahu   a bf:FunctionBlock.
} WHERE {
  {?ahu   bf:hasInput ?point.}
  UNION
  {?ahu   bf:hasOutput ?point.}
}
''')
print(len(g))

18


### Defining the Control Flow

The top of the AHU graph is showing the control flow. To create dynamically, we need new instances for the control function block and the control command signals. Therefore, we are dynamically creating new URI for the sub-function blocks and internal signals (via the BIND, CONCAT, and STR functions). Then we link it all together.

In [6]:
qres = g.update('''
PREFIX  bf:  <http://buildsys.org/ontologies/BrickFrame#>
PREFIX  ts:  <http://buildsys.org/ontologies/Brick#>
INSERT {
  ?controlURI  a  bf:FunctionBlock, ts:Controler.
  ?ahu         bf:hasPart    ?controlURI.
  ?coolerCmd   a  bf:Signal, ts:Cooling_Command.
  ?heaterCmd   a  bf:Signal, ts:Heating_Command.
  ?controlURI  bf:hasInput   ?sp, ?val.
  ?controlURI  bf:hasOutput  ?coolerCmd, ?heaterCmd.
} WHERE {
  ?ahu   a ts:AHU, bf:FunctionBlock.
  ?sp    a ts:Discharge_Air_Temperature_Setpoint.
  ?val   a ts:Discharge_Air_Temperature_Sensor.
  ?ahu   bf:hasPoint  ?sp, ?val.
  BIND(IRI(CONCAT(STR(?ahu),'_Controler')) as ?controlURI ).
  BIND(IRI(CONCAT(STR(?ahu),'_CoolerCmd')) as ?coolerCmd ).
  BIND(IRI(CONCAT(STR(?ahu),'_HeaterCmd')) as ?heaterCmd ).
}
''')
print(len(g))

29


### Defining the Medium Flow

We can create the medium flow at the bottom of the AHU graph in a similar manner. We create the Mixer, Cooler, and Heater and all internal signals with SPARUD trough:

In [7]:
qres = g.update('''
PREFIX  bf:  <http://buildsys.org/ontologies/BrickFrame#>
PREFIX  ts:  <http://buildsys.org/ontologies/Brick#>
INSERT {
  ?mixerURI    a  bf:FunctionBlock, ts:Mixed.
  ?coolerURI   a  bf:FunctionBlock, ts:Cooling.
  ?heaterURI   a  bf:FunctionBlock, ts:Heating.
  ?ahu         bf:hasPart    ?mixerURI.
  ?ahu         bf:hasPart    ?coolerURI.
  ?ahu         bf:hasPart    ?heaterURI.
  ?mixerTmp    a  bf:Signal, ts:Mixed_Air_Temperature_Sensor.
  ?coolerTmp   a  bf:Signal.
  ?mixerURI    bf:hasInput   ?oat, ?rat.
  ?mixerURI    bf:hasOutput  ?mixerTmp.
  ?coolerURI   bf:hasInput   ?mixerTmp.
  ?coolerURI   bf:hasOutput  ?coolerTmp.
  ?heaterURI   bf:hasInput   ?coolerTmp.
  ?heaterURI   bf:hasOutput  ?dat.
} WHERE {
  ?ahu   a ts:AHU, bf:FunctionBlock.
  ?rat   a ts:Return_Air_Temperature_Sensor.
  ?oat   a ts:Outside_Air_Temperature_Sensor.
  ?dat   a ts:Discharge_Air_Temperature_Sensor.
  ?ahu   bf:hasPoint  ?rat.
  ?ahu   bf:hasPoint  ?oat.
  ?ahu   bf:hasPoint  ?dat.
  BIND(IRI(CONCAT(STR(?ahu),'_Mixed'))     as ?mixerURI ).
  BIND(IRI(CONCAT(STR(?ahu),'_Cooling'))   as ?coolerURI ).
  BIND(IRI(CONCAT(STR(?ahu),'_Heating'))   as ?heaterURI ).
  BIND(IRI(CONCAT(STR(?ahu),'_MixerTmp'))  as ?mixerTmp ).
  BIND(IRI(CONCAT(STR(?ahu),'_CoolerTmp')) as ?coolerTmp ).
}
''')
print(len(g))

48


We finally have an ontology defining a full AHU

In [8]:
print(str(g.serialize(format='n3')).replace('\\n','\n'))

b'@prefix : <http://buildsys.org/ontologies/Example/AHU_FunctionBlock#> .
@prefix bf: <http://buildsys.org/ontologies/BrickFrame#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix ts: <http://buildsys.org/ontologies/Brick#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:AHU1 a ts:AHU,
        bf:FunctionBlock ;
    bf:hasInput :AHU1DATSP,
        :AHU1RAT,
        :OAT ;
    bf:hasOutput :AHU1DAT ;
    bf:hasPart :AHU1_Controler,
        :AHU1_Cooler,
        :AHU1_Heater,
        :AHU1_Mixer ;
    bf:hasPoint :AHU1DAT,
        :AHU1DATSP,
        :AHU1RAT,
        :OAT .

:AHU1_Controler a ts:Controler,
        bf:FunctionBlock ;
    bf:hasInput :AHU1DAT,
        :AHU1DATSP ;
    bf:hasOutput :AHU1_CoolerCmd,
        :AHU1_HeaterCmd .

:AHU1_Cooler a ts:Cooler,
        bf:FunctionBlock ;
    bf:hasInput :AHU1_MixerTmp ;
    bf:hasOutput :AHU1_Cool